### EM + General

ELBO derivation and related identities, for any density $q_n$.

\begin{align}
\log p(\theta, \mathbf{X})
&=
\log p(\theta) + \sum_n \log p(x_n|\theta) & (1)\\
&=
\log p(\theta) + \sum_n \log E_{q_n(z_n)}[p(x_n|z_n, \theta)] & (2) \\
&\geq 
\log p(\theta) + \sum_n E_{q_n(z_n)}[\log p(x_n|z_n, \theta)] & (3) \\
&=
\log p(\theta) + \sum_n E_{q_n(z_n)}\bigg[\log \frac{p(x_n, z_n|\theta)}{q_n(z_n)}\bigg] & (4) \\
&=
\log p(\theta) + \sum_n E_{q_n(z_n)}[\log p(x_n, z_n| \theta) - \log q_n(z_n)] & (5)\\
&=
\log p(\theta) + \sum_n E_{q_n(z_n)}[\log p(x_n| \theta) + \log p(z_n | x_n, \theta) - \log q_n(z_n)] & (6)\\
&=
\log p(\theta) + \sum_n  E_{q_n(z_n)}[\log p(x_n| \theta)] + E_{q_n(z_n)}\log p(z_n | x_n, \theta) - \log q_n(z_n)]  & (7)\\
&=
\log p(\theta) + \sum_n \log p(x_n| \theta) - D_{KL}(q_n(z_n) || p(z_n | x_n, \theta)) & (8)\\
&=
\log p(\theta, \mathbf{X}) - D_{KL}(q_n(z_n) || p(z_n | x_n, \theta)) & (9)\\
&= 
\underbrace{\mathcal{L}(\theta, q)}_{``ELBO''} & (10)
\end{align}

Notes: 

- EM is coordinate ascent on the ELBO.

- E-step: maximize ELBO w.r.t. $q$ (the hidden/approximating thing)

- M-step: maximize ELBO w.r.t. $\theta$ (model params). 

- (5) is a common starting point

- Look at (8) hard enough and you see the objective for amortized VI.

- When $q_n = p(z_n | x_n, \theta)$, the ELBO is tight and the "$\geq$" in line 3 becomes a "$=$". Proof:
\begin{align}
\mathcal{L}(\theta, q)
& = 
\log p(\theta) + \sum_n E_{q_n(z_n)}[\log p(x_n, z_n| \theta) - \log q_n(z_n)]\\
&=
\log p(\theta) + \sum_n E_{p(z_n | x_n, \theta)}[\log p(x_n, z_n| \theta) - p(z_n | x_n, \theta)]\\
&=
\log p(\theta) + \sum_n E_{p(z_n | x_n, \theta)}\bigg[\log \frac{p(x_n, z_n | \theta)}{p(z_n|x_n, \theta)}\bigg] \\
&=
\log p(\theta) \sum_n E_{p(z_n | x_n, \theta)}\log p(x_n | z_n, \theta) \\
&=
\log p(\theta) \sum_n \log p(x_n | \theta) \\
&=
\log p(\theta, \mathbf{X}).
\end{align}


### Mean-Field Approximation

First, helpful to remember
$$
E_{q_{1:m}}[\log q(z_{1:m})] = \sum_{i}^m E_{q_i}[\log q(z_i)]
$$

Try to smush together:

\begin{align}
D_{KL}(q(\theta; \lambda)||p(\theta|x))
&=
E_{q(\theta; \lambda)}[\log q(\theta; \lambda)] - E_{q(\theta; \lambda)}[\log p(\theta | \mathbf{X})] \\
&=
E_{q(\theta; \lambda)}[\log q(\theta; \lambda)] - E_{q(\theta; \lambda)}[\log p(\theta , \mathbf{X}) - \log p(\mathbf{X})]  \\
&=
E_{q(\theta; \lambda)}[\log q(\theta; \lambda)] - E_{q(\theta; \lambda)}[\log p(\theta , \mathbf{X})] + E_{q(\theta; \lambda)}[\log p(\mathbf{X})]  \\
&=
\underbrace{E_{q(\theta; \lambda)}[\log q(\theta; \lambda)] - E_{q(\theta; \lambda)}[\log p(\theta , \mathbf{X})]}_{-ELBO(\lambda) := \mathcal{L}(\lambda)} + \log p(\mathbf{X}).
\end{align}

Hence, zooming in on $\mathcal{L}(\lambda)$, we have by the mean-field assumptions, we have:

\begin{align}
\mathcal{L}(\lambda)
&=
E_{q(\theta; \lambda)}[\log p(\theta , \mathbf{X})] - 
E_{q(\theta; \lambda)}[\log q(\theta; \lambda)] \\
&=
E_{\prod_{i=1}^N q(\theta_i; \lambda_i)}[\log p(\theta , \mathbf{X})] - 
E_{\prod_{i=1}^N q(\theta_i; \lambda_i)}[\log q(\theta; \lambda)] \\
&=
E_{\prod_{i=1}^N q(\theta_i; \lambda_i)}[p(\theta, \mathbf{X})] - 
E_{\prod_{i=1}^N q(\theta_i; \lambda_i)}\bigg[\log \prod_{i=1}^N q(\theta_i; \lambda_i)\bigg] \\
&=
E_{\prod_{i=1}^N q(\theta_i; \lambda_i)}[\log p(\theta , \mathbf{X})] 
- 
E_{\prod_{i=1}^N q(\theta_i; \lambda_i)}\bigg[\sum_{i=1}^N \log q(\theta_i; \lambda_i)\bigg]
\end{align}

Then, for a particular $j$, we can look at

\begin{align}
...&=
E_{q(\theta_j; \lambda_j) \prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta , \mathbf{X})] - 
E_{q(\theta_j; \lambda_j) \prod_{i\neq j} q(\theta_i; \lambda_i)}\bigg[\sum_{i=1}^N \log q(\theta_i; \lambda_i)\bigg]\\
&=
E_{q(\theta_j; \lambda_j) \prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta_j |\theta_{i \neq j}, \mathbf{X}) + \log p(\theta_{i \neq j}, \mathbf{X})] - \\&\hspace{1.5cm}
E_{q(\theta_j; \lambda_j) \prod_{i\neq j} q(\theta_i; \lambda_i)}\bigg[\sum_{i=1}^N \log q(\theta_i; \lambda_i)\bigg]\\
&=
E_{q(\theta_j; \lambda_j) \prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta_j |\theta_{i \neq j}, \mathbf{X}) + \log p(\theta_{i \neq j}, \mathbf{X})] - \\&\hspace{1.5cm}
E_{q(\theta_j; \lambda_j)}[\log q(\theta_j; \lambda_j)] +  E_{\prod_{i\neq j} q(\theta_i; \lambda_i)}\bigg[\sum_{i\neq j} \log q(\theta_i; \lambda_i)\bigg]\\
&=
E_{q(\theta_j; \lambda_j) \prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta_j |\theta_{i \neq j}, \mathbf{X}) + \log p(\theta_{i \neq j}, \mathbf{X})] - 
E_{q(\theta_j; \lambda_j)}[\log q(\theta_j; \lambda_j)] +  c\\
&=
E_{q(\theta_j; \lambda_j)}E_{\prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta_j |\theta_{i \neq j}, \mathbf{X}) + \log p(\theta_{i \neq j}, \mathbf{X})] - 
E_{q(\theta_j; \lambda_j)}[\log q(\theta_j; \lambda_j)] +  c\\
&=
E_{q(\theta_j; \lambda_j)}[E_{\prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta_j |\theta_{i \neq j}, \mathbf{X})]] - 
E_{q(\theta_j; \lambda_j)}[\log q(\theta_j; \lambda_j)] +  c'.\\
\end{align}

Note that the mean-field independence assumption allows us to separate/stack up the expectations as we did. Then, if we call (up to additive constant)

$$
\log \tilde p(\theta_j) = E_{\prod_{i\neq j} q(\theta_i; \lambda_i)}[\log p(\theta_j |\theta_{i \neq j}, \mathbf{X})],
$$

the above algebra simplifies to 

$$
-D_{KL}(q(\theta_j; \lambda_j) || \tilde p(\theta_j)) + c'.
$$

So to minimize KL for a particular $\theta_j, \lambda_j$ pair, you choose

$$
q(\theta_j; \lambda_j) = \tilde p(\theta_j).
$$

Hence a CAVI step means setting equal to the full conditional, analagous to a Gibbs sampler.
